# Rigid registration

This example should show you how to perform rigid registration between two SIRF images.

SIRF's registration/resampling functionality is provided by wrapping and extending the [NiftyReg](http://cmictig.cs.ucl.ac.uk/wiki/index.php/NiftyReg) code base. Rigid and affine registrations are performed using NiftyReg's symmetric `aladin` algorithm, whereas non-rigid registrations use the symmetric `f3d` algorithm.

Although the example of rigid registration is given here, it is trivial to modify it for affine registrations and only slightly trickier to extend to non-rigid registration.

The images to be registered in this example are `test.nii.gz` and `test2.nii.gz`, which are two T1-weighted MR brain scans taken one year apart.

N.B.: Registration packages use different names for the sets of images they are registering. In NiftyReg (and therefore SIRF), the floating image is moved to match the reference image. In other packages, the floating=moving and reference=fixed. 

## Copyright

Author: Richard Brown
First version: 3rd April 2019

CCP PETMR Synergistic Image Reconstruction Framework (SIRF)
Copyright 2019 University College London.

This is software developed for the Collaborative Computational Project in Positron Emission Tomography and Magnetic Resonance imaging (http://www.ccppetmr.ac.uk/).

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.


## Initial set up

In [ ]:
# Standard stuff
import numpy
import matplotlib.pyplot as plt
%matplotlib notebook

# SIRF stuff
from sirf.Utilities import examples_data_path
import sirf.Reg as Reg
examples_path = examples_data_path('Registration')

In [ ]:
#%% First define some handy function definitions
# To make subsequent code cleaner, we have a few functions here. You can ignore
# ignore them when you first see this demo.
# They have (minimal) documentation using Python docstrings such that you 
# can do for instance "help(imshow)"
#
# First a function to display an image

def imshow(image, title=''):
    """Display an image with a colourbar, returning the plot handle. 
    
    Arguments:
    image -- a 2D array of numbers
    limits -- colourscale limits as [min,max]. An empty [] uses the full range
    title -- a string for the title of the plot (default "")
    """
    plt.title(title)
    bitmap=plt.imshow(image)
    limits=[numpy.nanmin(image),numpy.nanmax(image)]
                
    plt.clim(limits[0], limits[1])
    plt.colorbar(shrink=.6)
    plt.axis('off');
    return bitmap

## Engine for loading images

By default this example uses `pReg` as the engine to open the images, which handles NIfTI images. 

You might want to register different types of images - perhaps your floating image is a STIR interfile? If so, change the second line such that it reads `import pSTIR as eng_flo`.

In [ ]:
# In this example, we will use pReg as the engine to open our images
import sirf.Reg as eng_ref
import sirf.Reg as eng_flo

## Open and display the images

In [ ]:
# Open the images
ref_file = examples_path + "/test.nii.gz"
flo_file = examples_path + "/test2.nii.gz"
ref = eng_ref.ImageData(ref_file)
flo = eng_flo.ImageData(flo_file)

In [ ]:
# Display the images
ref_slice = int(ref.get_dimensions()[1]/2);
flo_slice = int(flo.get_dimensions()[1]/2);
plt.subplot(1,2,1);
imshow(ref.as_array()[ref_slice,:,:], 'Reference image, slice: %i' % int(ref_slice));
plt.subplot(1,2,2);
imshow(flo.as_array()[flo_slice,:,:], 'Floating image, slice: %i' % int(flo_slice));

## Set up the registration object

In [ ]:
# Set to NiftyF3dSym for non-rigid
algo = Reg.NiftyAladinSym()

# Set images
algo.set_reference_image(ref)
algo.set_floating_image(flo)

# What else can we do?
help(algo)

## Setting parameters

From the help above, it looks like we can set registration parameters both via a file and directly. Let's try both.

In [ ]:
# Setting via parameter file
par_file = examples_path + "/paramFiles/niftyreg_aladin.par"
algo.set_parameter_file(par_file)

algo.set_parameter('SetPerformRigid','1')
algo.set_parameter('SetPerformAffine','0')
#algo.set_parameter('SetWarpedPaddingValue','0') <- NaN by default, uncomment to set to 0.

## Register!

In [ ]:
algo.process()

## Show the registered image

The registered image will be the same size as the reference image, so we can use `ref_slice`.

In [ ]:
output = algo.get_output()
output_arr = output.as_array()
imshow(output_arr[ref_slice,:,:], 'Registered image, slice: %i' % int(ref_slice));

## Show the transformation matrix

In [ ]:
TM = algo.get_transformation_matrix_forward()
print(TM.as_array())

## Deformation field
The deformation field will be a 3D tensor image, where the components are the deformation in the x-, y- and z-directions, respectively.

In the NIfTI format, the number of voxels in each of the dimensions is shown as a 8-dimensional array. The elements represent the following:
```
dim[0] = number of dimensions
dim[1] = x
dim[2] = y
dim[3] = z
dim[4] = t (time)
dim[5] = u (tensor component)
dim[6] = v
dim[7] = w
```
For a deformation field, the tensor information is stored in the `u` dimension. So we would expect `deformation.get_dimensions()` to give `[5 x y z 1 3 1 1]` (5 because the 5th dimension is the last non-singleton dimension).

In [ ]:
deformation = algo.get_deformation_field_forward()
print("Deformation NIfTI dimensions")
print(deformation.get_dimensions())
print("Deformation NIfTI dimensions as a numpy array (using shape)")
def_arr = deformation.as_array()
print(def_arr.shape)
plt.subplot(3,1,1);
imshow(deformation.as_array()[ref_slice,:,:,0,0], 'Deformation field x-direction, slice: %i' % int(ref_slice));
plt.subplot(3,1,2);
imshow(deformation.as_array()[ref_slice,:,:,0,1], 'Deformation field y-direction, slice: %i' % int(ref_slice));
plt.subplot(3,1,3);
imshow(deformation.as_array()[ref_slice,:,:,0,2], 'Deformation field z-direction, slice: %i' % int(ref_slice));

## Displacement field

The displacement field is mostly the same as the deformation field, and the x-, y- and z-components can be displayed.

#### What's the difference between displacement and deformation fields?
From an input image, <b>`x`</b>, the warped image, <b>`W`</b>, can be generated with either a deformation or displacement field.
The formulae for the two are given below:
<pre>
<b>W</b>(<b>x</b>) = <b>x</b> + disp(<b>x</b>)
<b>W</b>(<b>x</b>) = def(<b>x</b>)
</pre>

In [ ]:
displacement = algo.get_displacement_field_forward()
plt.subplot(3,1,1);
imshow(displacement.as_array()[ref_slice,:,:,0,0], 'Displacement field x-direction, slice: %i' % int(ref_slice));
plt.subplot(3,1,2);
imshow(displacement.as_array()[ref_slice,:,:,0,1], 'Displacement field y-direction, slice: %i' % int(ref_slice));
plt.subplot(3,1,3);
imshow(displacement.as_array()[ref_slice,:,:,0,2], 'Displacement field z-direction, slice: %i' % int(ref_slice));